In [11]:
import pandas as pd

# Load your dataset
df = pd.read_csv('supermarket_sales.csv')

# Clean column names by stripping extra spaces
df.columns = df.columns.str.strip()

# Create Customers DataFrame (unique values for customers)
customers_df = df[['Gender', 'Customer type']].drop_duplicates()
customers_df['CustomerID'] = range(1, len(customers_df) + 1)
customers_df.to_csv('customers.csv', index=False)

# Create Branches DataFrame (unique values for branches)
branches_df = df[['Branch', 'City']].drop_duplicates()
branches_df['BranchID'] = range(1, len(branches_df) + 1)
branches_df.to_csv('branches.csv', index=False)

# Create Products DataFrame (unique values for products)
products_df = df[['Product line', 'Unit price']].drop_duplicates()
products_df['ProductID'] = range(1, len(products_df) + 1)

# Drop any duplicate product lines to ensure uniqueness
products_df = products_df.drop_duplicates(subset=['Product line'])

# Check the DataFrame after dropping duplicates
print("Products DataFrame:\n", products_df.head())

# Create Transactions DataFrame with required columns and matching IDs
transactions_df = df.copy()

# Map CustomerID using both 'Gender' and 'Customer type' to ensure uniqueness
transactions_df['CustomerID'] = transactions_df.apply(
    lambda row: customers_df[(customers_df['Gender'] == row['Gender']) & 
                             (customers_df['Customer type'] == row['Customer type'])]['CustomerID'].values[0], 
    axis=1
)

# Map BranchID using Branch column
transactions_df['BranchID'] = transactions_df['Branch'].map(
    branches_df.set_index('Branch')['BranchID']
)

# Map ProductID using Product line column
transactions_df['ProductID'] = transactions_df['Product line'].map(
    products_df.set_index('Product line')['ProductID']
)

# Check the columns before renaming
print("Transactions DataFrame columns before renaming:", transactions_df.columns)

# Selecting relevant columns and renaming them for transactions
transactions_df = transactions_df[[
    'Invoice ID', 'CustomerID', 'BranchID', 'ProductID', 'Quantity', 'Tax 5%', 'Total', 'Date', 'Time', 'Payment', 'cogs',
    'gross margin percentage', 'gross income', 'Rating'
]]

# Verify the number of columns
print("Columns after selection:", transactions_df.columns)

# Adjust column names to match the number of columns
transactions_df.columns = [
    'InvoiceID', 'CustomerID', 'BranchID', 'ProductID', 'Quantity', 'Tax', 'Total', 'Date', 'Time', 'Payment', 'Cogs', 
    'GrossMarginPercentage', 'GrossIncome', 'Rating'
]

# Save the final DataFrame to a CSV
transactions_df.to_csv('transactions.csv', index=False)


Products DataFrame:
              Product line  Unit price  ProductID
0       Health and beauty       74.69          1
1  Electronic accessories       15.28          2
2      Home and lifestyle       46.33          3
4       Sports and travel       86.31          5
9      Food and beverages       54.84         10
Transactions DataFrame columns before renaming: Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating', 'CustomerID', 'BranchID', 'ProductID'],
      dtype='object')
Columns after selection: Index(['Invoice ID', 'CustomerID', 'BranchID', 'ProductID', 'Quantity',
       'Tax 5%', 'Total', 'Date', 'Time', 'Payment', 'cogs',
       'gross margin percentage', 'gross income', 'Rating'],
      dtype='object')


In [16]:
import pymysql
from pymysql import Error

# Initialize connection variable
connection = None

# Establish the connection to the MySQL database
try:
    connection = pymysql.connect(
        host='localhost',
        user='root',  # Replace with your MySQL username
        password='amn@#2005',  # Replace with your MySQL password
        database='supermarket',  # Replace with your database name
        cursorclass=pymysql.cursors.DictCursor  # Optional: to get results as dictionaries
    )

    if connection.open:
        print("Connected to the database")

        cursor = connection.cursor()

        # 1. Inserting data into 'customers' table
        customers_insert_query = """
        INSERT INTO customers (CustomerID, Gender, CustomerType) 
        VALUES (%s, %s, %s)
        """
        for index, row in customers_df.iterrows():
            cursor.execute(customers_insert_query, (row['CustomerID'], row['Gender'], row['Customer type']))
        
        # Commit the transaction for customers table
        connection.commit()
        print(f"Inserted {len(customers_df)} records into the customers table.")

        # 2. Inserting data into 'branches' table
        branches_insert_query = """
        INSERT INTO branches (BranchID, BranchName, City)
        VALUES (%s, %s, %s)
        """
        for index, row in branches_df.iterrows():
            cursor.execute(branches_insert_query, (row['BranchID'], row['Branch'], row['City']))
        
        # Commit the transaction for branches table
        connection.commit()
        print(f"Inserted {len(branches_df)} records into the branches table.")

        # 3. Inserting data into 'products' table
        products_insert_query = """
        INSERT INTO products (ProductID, ProductLine, UnitPrice)
        VALUES (%s, %s, %s)
        """
        for index, row in products_df.iterrows():
            cursor.execute(products_insert_query, (row['ProductID'], row['Product line'], row['Unit price']))
        
        # Commit the transaction for products table
        connection.commit()
        print(f"Inserted {len(products_df)} records into the products table.")

        # 4. Inserting data into 'transactions' table
        transactions_insert_query = """
        INSERT INTO transactions (InvoiceID, CustomerID, BranchID, ProductID, Quantity, Tax, Total, DateTime, Payment, Cogs, GrossMarginPercentage, GrossIncome, Rating)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        for index, row in transactions_df.iterrows():
            date_time = f"{row['Date']} {row['Time']}"  # Combining 'Date' and 'Time' columns into a single datetime column
            cursor.execute(transactions_insert_query, (
                row['InvoiceID'], row['CustomerID'], row['BranchID'], row['ProductID'], row['Quantity'],
                row['Tax 5%'], row['Total'], date_time, row['Payment'], row['cogs'],
                row['gross margin percentage'], row['gross income'], row['Rating']
            ))
        
        # Commit the transaction for transactions table
        connection.commit()
        print(f"Inserted {len(transactions_df)} records into the transactions table.")

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    # Closing the cursor and connection if they exist
    if connection and connection.open:
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Connected to the database
Error while connecting to MySQL (1062, "Duplicate entry '1' for key 'customers.PRIMARY'")
MySQL connection is closed


In [15]:
print(transactions_df.columns)


Index(['InvoiceID', 'CustomerID', 'BranchID', 'ProductID', 'Quantity', 'Tax',
       'Total', 'Date', 'Time', 'Payment', 'Cogs', 'GrossMarginPercentage',
       'GrossIncome', 'Rating'],
      dtype='object')


In [22]:
import pymysql

# Connect to the MySQL database
try:
    connection = pymysql.connect(
        host='localhost',
        user='root',  # Replace with your MySQL username
        password='amn@#2005',  # Replace with your MySQL password
        database='supermarket'  # Replace with your database name
    )

    if connection.open:
        print("Connected to the database")
        
        # Create a cursor
        cursor = connection.cursor()

        # Insert data into 'customers' table (with INSERT IGNORE to avoid duplicate entries)
        customers_insert_query = """
        INSERT IGNORE INTO customers (CustomerID, Gender, CustomerType) 
        VALUES (%s, %s, %s)
        """
        for index, row in customers_df.iterrows():
            cursor.execute(customers_insert_query, (row['CustomerID'], row['Gender'], row['Customer type']))

        # Commit the transaction for customers table
        connection.commit()
        print(f"Inserted {len(customers_df)} records into the customers table.")

        # Insert data into 'branches' table (with INSERT IGNORE)
        branches_insert_query = """
        INSERT IGNORE INTO branches (BranchID, BranchName, City)
        VALUES (%s, %s, %s)
        """
        for index, row in branches_df.iterrows():
            cursor.execute(branches_insert_query, (row['BranchID'], row['Branch'], row['City']))
        
        # Commit the transaction for branches table
        connection.commit()
        print(f"Inserted {len(branches_df)} records into the branches table.")

        # Insert data into 'products' table (with INSERT IGNORE)
        products_insert_query = """
        INSERT IGNORE INTO products (ProductID, ProductLine, UnitPrice)
        VALUES (%s, %s, %s)
        """
        for index, row in products_df.iterrows():
            cursor.execute(products_insert_query, (row['ProductID'], row['Product line'], row['Unit price']))
        
        # Commit the transaction for products table
        connection.commit()
        print(f"Inserted {len(products_df)} records into the products table.")

        # Insert data into 'transactions' table (with INSERT IGNORE)
        transactions_insert_query = """
        INSERT IGNORE INTO transactions (InvoiceID, CustomerID, BranchID, ProductID, Quantity, Tax, Total, DateTime, Payment, Cogs, GrossMarginPercentage, GrossIncome, Rating)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        for index, row in transactions_df.iterrows():
            date_time = f"{row['Date']} {row['Time']}"  # Combining 'Date' and 'Time' columns into a single datetime column
            cursor.execute(transactions_insert_query, (
                row['InvoiceID'], row['CustomerID'], row['BranchID'], row['ProductID'], row['Quantity'],
                row['Tax'], row['Total'], date_time, row['Payment'], row['Cogs'],
                row['GrossMarginPercentage'], row['GrossIncome'], row['Rating']
            ))
        
        # Commit the transaction for transactions table
        connection.commit()
        print(f"Inserted {len(transactions_df)} records into the transactions table.")

except pymysql.MySQLError as e:
    print("Error while connecting to MySQL:", e)

finally:
    # Ensure cursor and connection are properly closed after all operations
    try:
        if connection.open:
            cursor.close()
            connection.close()
            print("MySQL connection is closed.")
    except NameError:
        print("Connection was not established.")


Connected to the database
Inserted 4 records into the customers table.
Inserted 3 records into the branches table.
Inserted 6 records into the products table.
Inserted 1000 records into the transactions table.
MySQL connection is closed.
